In [1]:
class Rx:
    "has the nums of a treatment, its name and rank"
    def __init__(i,lst):
        i.rx, i.lst = lst[0], lst[1:]
        i.mean = sum(i.lst)/len(i.lst)
        i.rank = 0
    def __repr__(i):
        return 'rank #%s %s at %s'%(i.rank,i.rx,i.mean)

def a12s(lst,rev=True,enough=0.66):
    "sees if lst[i+1] has rank higher than lst[i]"
    lst = [Rx(one) for one in lst]
    lst = sorted(lst,key=lambda x:x.mean,reverse=rev)
    one   = lst[0]
    rank = one.rank = 1
    for two in lst[1:]:
        if a12(one.lst,two.lst,rev) > enough: rank += 1
        two.rank = rank
        one = two
    return lst

def a12(lst1,lst2,rev=True):
    "how often is x in lst1 more than y in lst2?"
    more = same = 0.0
    for x in lst1:
        for y in lst2:
            if   x==y : same += 1
            elif rev     and x > y : more += 1
            elif not rev and x < y : more += 1
    return (more + 0.5*same)  / (len(lst1)*len(lst2))

def fromFile(f="a12.dat",rev=True,enough=0.66):
    "utility for reading sample data from disk"
    import re
    cache = {}
    num, space = r'^\+?-?[0-9]', r'[ \t\n]+'
    for line in open(f):
        line = line.strip()
        if line:
            for word in re.split(space,line):
                if re.match(num,word[0]):
                    cache[now] += [float(word)]
                else:
                    now  = word
                    cache[now] = [now]
    return a12s(cache.values(),rev,enough)

In [2]:
# Load dataframe with results
import pandas as pd

df = pd.read_csv('edge_roc_df.csv')
df

,Unnamed: 0,run,acdc,node_sp,edge_sp,eap,integrated_grad
0,0,11,1.000000,0.926667,1.000000,0.000000,1.000000
1,1,13,0.978723,0.851064,0.978723,0.936170,0.985816
2,2,18,0.978723,0.595745,0.886525,0.326241,0.978723
3,3,19,0.940000,0.900000,0.860000,0.000000,0.900000
4,4,20,0.921569,0.725490,0.529412,0.000000,0.725490
5,5,21,0.796086,0.705808,0.715909,0.594066,0.721591
6,6,26,1.000000,0.960784,1.000000,0.000000,0.980392
7,7,29,0.441176,0.176471,0.450980,0.000000,0.411765
8,8,3,0.979592,0.933673,1.000000,0.989796,0.989796
9,9,33,0.794118,0.852941,0.823529,0.000000,0.911765


In [3]:
import scipy.stats as stats

def run_stats(alg_1, alg_2):
    x = df[alg_1]
    y = df[alg_2]
    wilcoxon_result = stats.wilcoxon(x, y)
    print(wilcoxon_result)
    wilcoxon_p_values_df.loc[alg_1, alg_2] = wilcoxon_result.pvalue
    
    a12_result = a12(x, y)
    print(a12_result)
    a12_values_df.loc[alg_1, alg_2] = a12_result
    
available_algs = list(df.columns[2:])
print(available_algs)

# for each combination of algorithms, run the stats, and save it in a DF
wilcoxon_p_values_df = pd.DataFrame(index=available_algs, columns=available_algs)
a12_values_df = pd.DataFrame(index=available_algs, columns=available_algs)
for i in range(len(available_algs)):
    for j in range(i+1, len(available_algs)):
        print(f'{available_algs[i]} vs {available_algs[j]}')
        run_stats(available_algs[i], available_algs[j])

['acdc', 'node_sp', 'edge_sp', 'eap', 'integrated_grad']
acdc vs node_sp
WilcoxonResult(statistic=6.0, pvalue=0.00042724609375)
0.7421875
acdc vs edge_sp
WilcoxonResult(statistic=6.0, pvalue=0.02841686417486375)
0.541015625
acdc vs eap
WilcoxonResult(statistic=1.0, pvalue=6.103515625e-05)
0.91015625
acdc vs integrated_grad
WilcoxonResult(statistic=18.0, pvalue=0.09948088075636445)
0.5546875
node_sp vs edge_sp
WilcoxonResult(statistic=22.0, pvalue=0.0155029296875)
0.35546875
node_sp vs eap
WilcoxonResult(statistic=3.0, pvalue=0.000152587890625)
0.84375
node_sp vs integrated_grad
WilcoxonResult(statistic=0.0, pvalue=0.000978706525317055)
0.31640625
edge_sp vs eap
WilcoxonResult(statistic=0.0, pvalue=3.0517578125e-05)
0.88671875
edge_sp vs integrated_grad
WilcoxonResult(statistic=26.0, pvalue=0.3078214719088427)
0.486328125
eap vs integrated_grad
WilcoxonResult(statistic=0.0, pvalue=0.0006483852534615972)
0.111328125


In [4]:
wilcoxon_p_values_df = wilcoxon_p_values_df.round(3)
wilcoxon_p_values_df

,acdc,node_sp,edge_sp,eap,integrated_grad
acdc,NaN,0.000427,0.028417,0.000061,0.099481
node_sp,NaN,NaN,0.015503,0.000153,0.000979
edge_sp,NaN,NaN,NaN,0.000031,0.307821
eap,NaN,NaN,NaN,NaN,0.000648
integrated_grad,NaN,NaN,NaN,NaN,NaN


In [9]:
# set precision to 3
a12_values_df = a12_values_df.round(3)
# set print options todisplay precision to 3
pd.set_option('display.precision', 3)
a12_values_df

,acdc,node_sp,edge_sp,eap,integrated_grad
acdc,NaN,0.742,0.541,0.91,0.555
node_sp,NaN,NaN,0.355,0.844,0.316
edge_sp,NaN,NaN,NaN,0.887,0.486
eap,NaN,NaN,NaN,NaN,0.111
integrated_grad,NaN,NaN,NaN,NaN,NaN


In [10]:
df

,Unnamed: 0,run,acdc,node_sp,edge_sp,eap,integrated_grad
0,0,11,1.000,0.927,1.000,0.000,1.000
1,1,13,0.979,0.851,0.979,0.936,0.986
2,2,18,0.979,0.596,0.887,0.326,0.979
3,3,19,0.940,0.900,0.860,0.000,0.900
4,4,20,0.922,0.725,0.529,0.000,0.725
5,5,21,0.796,0.706,0.716,0.594,0.722
6,6,26,1.000,0.961,1.000,0.000,0.980
7,7,29,0.441,0.176,0.451,0.000,0.412
8,8,3,0.980,0.934,1.000,0.990,0.990
9,9,33,0.794,0.853,0.824,0.000,0.912
